## Professional Athlete Classifier Using Azure Machine Learning Service (Tensorflow 2.0)

The following tutorial demonstrates an end-to-end workflow for training and deploying a Tensorflow 2.0 model on Azure Machine Learning Service. The notebook will cover the following topics:

- Azure Machine Learning service workspace and other basic concepts
- Creating a dataset using Azure Cognitive Services and hosting it on a blob datastore
- Setting up an auto-scaling AML Compute cluster
- Training a distributed model across multiple nodes and GPUs
- Deploying a model and creating a web service endpoint


### Professional Athlete Classification Problem Statement

Professional sporting events have the power to bring friends, families, and co-workers together at home, at bars, and at the venue itself. While most people enjoy the social aspect and the entertainment of the game, few can recognize every single athlete that is on the court, field, or ice.

In this demo, we will be building a model that help solves this problem by classifying NBA players based upon their photo. While this can be extended to a multi-classification problem, this example will focus on a binary classification between two of the top NBA players today - Lebron James and Stephen Curry.

Lebron James           |  Stephen Curry
:---------------------:|:-------------------------:
![](./test/lebron.jpg) |  ![](./test/steph.jpg)

### What is Azure Machine Learning Service?
Azure Machine Learning service is a cloud service that you can use to develop and deploy machine learning models. Using Azure Machine Learning service, you can track your models as you build, train, deploy, and manage them, all at the broad scale that the cloud provides.
![](./images/aml-overview.png)

### Prerequisites

The tutorial requires the following prerequisites:

- An existing Azure Machine Learning service workspace
- An existing notebook VM instance inside the workspace

The tutorial is designed to be run on the notebook VM instance. Click [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-1st-experiment-sdk-setup) for instructions on creating a workspace and notebook VM.

### Connect To Workspace

The [workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace(class)?view=azure-ml-py) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. The workspace holds all your experiments, compute targets, models, datastores, etc.

You can [open ml.azure.com](https://ml.azure.com) to access your workspace resources through a graphical user interface of **Azure Machine Learning studio**.

![](./images/aml-workspace.png)

**You will be asked to login in the next step. Use your Microsoft AAD credentials.**

In [109]:
from azureml.core import Workspace

workspace = Workspace.from_config()
print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep = '\n')

Workspace name: john-east
Azure region: eastus
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: john


### Create Dataset Using Azure Cognitive Services

[Azure Cognitive Services](https://azure.microsoft.com/en-us/services/cognitive-services/) is a comprehensive family of AI services and cognitive APIs. We will be using the [Bing Image Search API](https://azure.microsoft.com/en-us/services/cognitive-services/bing-image-search-api/) to find and download images of professional athletes in order to build a dataset that can be used for training.

To complete this section, you will need to create an Azure Cognitive Service resource in order to obtain an API key. [Click here](https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Cwindows) for instructions on how to do so.

**Save the API key in a variable that we will use later.**

In [115]:
cognitive_service_api_key = '<API KEY HERE>'

We have prepared a script which does the following steps in sequence:

1) For each athlete, make a query to the Bing Image Search API for profile photos.
2) Download each image from the resulting response and crop out the face of the athlete.
3) Save the images to a data directory in corresponding test/valid and label folders.

**Let's take a look at the data preparation script**

In [114]:
%pycat src/cognitive-service-web-scraper.py

In [12]:
!pip install opencv-python imutils

In [5]:
datastore = workspace.get_default_datastore()

In [12]:
config_file_name = 'blobfuse_connection.cfg'

config_params = 'accountName {}\naccountKey {}\ncontainerName {}'.format(
    datastore.account_name, 
    datastore.account_key, 
    datastore.container_name)

config_file = open(config_file_name, 'w')
config_file.writelines(config_params)
config_file.close()

In [14]:
blobstore_dir = 'blobstore'

In [ ]:
import os
if not os.path.exists(blobstore_dir):
    os.makedirs(blobstore_dir)

In [15]:
!sudo blobfuse {blobstore_dir} --tmp-path=/mnt/resource/blobfusetmp  --config-file={config_file_name} -o attr_timeout=240 -o entry_timeout=240 -o negative_timeout=120

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [6]:
data_dir = 'nba-players'

In [17]:
!sudo mkdir {blobstore_dir}/{data_dir}

In [18]:
import sys

!sudo {sys.executable} cognitive-service-web-scraper.py --root_dir {blobstore_dir}/{data_dir} --image_dim 400

Writing image 0 for Lebron James...
Writing image 1 for Lebron James...
Writing image 2 for Lebron James...
Writing image 3 for Lebron James...
Writing image 4 for Lebron James...
Writing image 5 for Lebron James...
Writing image 6 for Lebron James...
Writing image 7 for Lebron James...
Writing image 8 for Lebron James...
Writing image 9 for Lebron James...
Writing image 10 for Lebron James...
Writing image 11 for Lebron James...
Writing image 12 for Lebron James...
Writing image 13 for Lebron James...
Writing image 14 for Lebron James...
Writing image 15 for Lebron James...
Writing image 16 for Lebron James...
Writing image 17 for Lebron James...
Writing image 18 for Lebron James...
Writing image 19 for Lebron James...
Writing image 20 for Lebron James...
Writing image 21 for Lebron James...
Writing image 22 for Lebron James...
Writing image 23 for Lebron James...
Writing image 24 for Lebron James...
Writing image 25 for Lebron James...
Writing image 26 for Lebron James...
Writing ima

Writing image 210 for Lebron James...
Writing image 211 for Lebron James...
Writing image 212 for Lebron James...
Writing image 213 for Lebron James...
Writing image 214 for Lebron James...
Writing image 215 for Lebron James...
Writing image 216 for Lebron James...
Writing image 217 for Lebron James...
Writing image 218 for Lebron James...
Writing image 219 for Lebron James...
Writing image 220 for Lebron James...
Writing image 221 for Lebron James...
Writing image 222 for Lebron James...
Writing image 223 for Lebron James...
Writing image 224 for Lebron James...
Writing image 225 for Lebron James...
Writing image 226 for Lebron James...
Writing image 227 for Lebron James...
Writing image 228 for Lebron James...
Writing image 229 for Lebron James...
Writing image 230 for Lebron James...
Writing image 231 for Lebron James...
Writing image 232 for Lebron James...
Writing image 233 for Lebron James...
Writing image 234 for Lebron James...
Writing image 235 for Lebron James...
Writing imag

Writing image 420 for Lebron James...
Writing image 421 for Lebron James...
Writing image 422 for Lebron James...
Writing image 423 for Lebron James...
Skipping blobstore/nba-players/valid/Lebron_James/423.jpg due to download error:
Writing image 423 for Lebron James...
Writing image 424 for Lebron James...
Writing image 425 for Lebron James...
Writing image 426 for Lebron James...
Writing image 427 for Lebron James...
Writing image 428 for Lebron James...
Writing image 429 for Lebron James...
Writing image 430 for Lebron James...
Writing image 431 for Lebron James...
Writing image 432 for Lebron James...
Writing image 433 for Lebron James...
Writing image 434 for Lebron James...
Writing image 435 for Lebron James...
Writing image 436 for Lebron James...
Writing image 437 for Lebron James...
Writing image 438 for Lebron James...
Writing image 439 for Lebron James...
Writing image 440 for Lebron James...
Writing image 441 for Lebron James...
Writing image 442 for Lebron James...
Writing

Writing image 128 for Stephen Curry...
Writing image 129 for Stephen Curry...
Writing image 130 for Stephen Curry...
Writing image 131 for Stephen Curry...
Writing image 132 for Stephen Curry...
Writing image 133 for Stephen Curry...
Writing image 134 for Stephen Curry...
Writing image 135 for Stephen Curry...
Writing image 136 for Stephen Curry...
Writing image 137 for Stephen Curry...
Writing image 138 for Stephen Curry...
Writing image 139 for Stephen Curry...
Writing image 140 for Stephen Curry...
Writing image 141 for Stephen Curry...
Writing image 142 for Stephen Curry...
Writing image 143 for Stephen Curry...
Writing image 144 for Stephen Curry...
Writing image 145 for Stephen Curry...
Writing image 146 for Stephen Curry...
Writing image 147 for Stephen Curry...
Writing image 148 for Stephen Curry...
Writing image 149 for Stephen Curry...
Writing image 150 for Stephen Curry...
Writing image 151 for Stephen Curry...
Writing image 152 for Stephen Curry...
Writing image 153 for Ste

Writing image 329 for Stephen Curry...
Writing image 330 for Stephen Curry...
Writing image 331 for Stephen Curry...
Writing image 332 for Stephen Curry...
Writing image 333 for Stephen Curry...
Writing image 334 for Stephen Curry...
Writing image 335 for Stephen Curry...
Writing image 336 for Stephen Curry...
Writing image 337 for Stephen Curry...
Writing image 338 for Stephen Curry...
Writing image 339 for Stephen Curry...
Writing image 340 for Stephen Curry...
Writing image 341 for Stephen Curry...
Writing image 342 for Stephen Curry...
Writing image 343 for Stephen Curry...
Writing image 344 for Stephen Curry...
Writing image 345 for Stephen Curry...
Writing image 346 for Stephen Curry...
Writing image 347 for Stephen Curry...
Writing image 348 for Stephen Curry...
Writing image 349 for Stephen Curry...
Writing image 350 for Stephen Curry...
Writing image 351 for Stephen Curry...
Writing image 352 for Stephen Curry...
Writing image 353 for Stephen Curry...
Writing image 354 for Ste

...

Cropping image: blobstore/nba-players/train/Stephen_Curry/81.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/338.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/182.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/203.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/181.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/45.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/0.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/132.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/340.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/29.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/113.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/139.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/187.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/150.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/34.jpg
Cropping image: 

Cropping image: blobstore/nba-players/train/Stephen_Curry/1.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/248.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/97.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/274.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/307.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/329.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/75.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/328.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/111.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/298.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/229.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/211.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/215.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/228.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/308.jpg
Cropping image

Cropping image: blobstore/nba-players/train/Lebron_James/178.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/179.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/18.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/180.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/181.jpg
Removing corrupted file: blobstore/nba-players/train/Lebron_James/181.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/182.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/183.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/184.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/185.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/186.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/187.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/188.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/189.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/19.jpg
Cropping image: bl

Cropping image: blobstore/nba-players/train/Lebron_James/288.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/289.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/29.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/290.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/291.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/292.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/293.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/294.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/295.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/296.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/297.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/298.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/299.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/3.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/30.jpg
Cropping image: blobstore/nba

Cropping image: blobstore/nba-players/train/Lebron_James/86.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/87.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/88.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/89.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/9.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/90.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/91.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/92.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/93.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/94.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/95.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/96.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/97.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/98.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/99.jpg


### Register Dataset

In [7]:
from azureml.core import Dataset

nba_players_dataset = Dataset.File.from_files(path=(datastore, data_dir))

nba_players_dataset = nba_players_dataset.register(workspace=workspace,
                                       name='NBA Players Dataset',
                                       description='Dataset containing pictures of NBA players')

In [8]:
nba_players_dataset = workspace.datasets['NBA Players Dataset']

### Create Compute Target

In [97]:
from azureml.core.compute import AmlCompute, ComputeTarget

cluster_name = 'k80cluster'
compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC12',
                                                       idle_seconds_before_scaledown=6000,
                                                       min_nodes=0, 
                                                       max_nodes=4)

compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [98]:
compute_target = workspace.compute_targets['k80cluster']

### Train

In [99]:
from azureml.core import Experiment

experiment_name = 'nba-player-classifier' 
experiment = Experiment(workspace, name=experiment_name)

In [106]:
from azureml.train.dnn import Mpi
from azureml.train.dnn import TensorFlow

distributed_estimator = TensorFlow(source_directory='src',
                                  entry_script='train_horovod.py',
                                  compute_target=compute_target,
                                  script_params = {
                                      '--data_dir': nba_players_dataset.as_named_input('nbaplayersdata').as_mount(),
                                      '--image_dim': 250,
                                      '--learning_rate': 0.001,
                                      '--batch_size': 16,
                                      '--steps_per_epoch': 100,
                                      '--num_epochs': 25,
                                      '--dropout_rate': 0.5,
                                      '--activation_function': 'softmax',
                                      '--output_dir':'./outputs',
                                  },
                                  framework_version='2.0',
                                  node_count=4,
                                  distributed_training=Mpi(process_count_per_node=2),
                                  use_gpu=True,
                                  pip_packages=['Pillow==6.2.0', 'scipy==1.1.0', 'azureml-dataprep[fuse,pandas]==1.1.29'])

distributed_run = experiment.submit(distributed_estimator)

In [112]:
from azureml.widgets import RunDetails
RunDetails(distributed_run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
model = best_run.register_model(model_name='nba-player-classifier', 
                                model_path='outputs',
                                datasets=[('train, test, validation data', nba_players_dataset)],
                                description='ResNet 50 model of for classifying NBA players')

In [62]:
from azureml.train.hyperdrive import GridParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice


param_sampling = GridParameterSampling( {
        '--dropout_rate': choice(0.3, 0.5, 0.7),
        '--activation_function': choice('softmax', 'relu', 'tanh')
    }
)

In [63]:
from azureml.train.hyperdrive import BanditPolicy
from azureml.train.hyperdrive import PrimaryMetricGoal

primary_metric_name='val_accuracy'
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE

early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=2)

In [64]:
from azureml.train.dnn import TensorFlow

estimator = TensorFlow(source_directory='src',
                       entry_script='train.py',
                       compute_target=compute_target,
                       script_params = {
                              '--data_dir': nba_players_dataset.as_named_input('nbaplayersdata').as_mount(),
                              '--image_dim': 250,
                              '--learning_rate': 0.001,
                              '--batch_size': 16,
                              '--steps_per_epoch': 100,
                              '--num_epochs': 25,
                              '--output_dir':'./outputs',
                        },
                        framework_version='2.0',
                        use_gpu=True,
                        pip_packages=['Pillow==6.2.0', 'scipy==1.1.0', 'azureml-dataprep[fuse,pandas]==1.1.29'])

In [65]:
from azureml.train.hyperdrive import HyperDriveConfig

hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                                         hyperparameter_sampling=param_sampling, 
                                         policy=early_termination_policy,
                                         primary_metric_name=primary_metric_name, 
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=10,
                                         max_concurrent_runs=2)

run = experiment.submit(hyperdrive_run_config)

In [95]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [94]:
run.cancel()

In [ ]:
model = best_run.register_model(model_name='nba-player-classifier', 
                                model_path='outputs',
                                datasets=[('train, test, validation data', nba_players_dataset)],
                                description='ResNet 50 model of for classifying NBA players')

### Deploy Model

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','pandas'],
                                 pip_packages=['inference-schema[numpy-support]', 'azureml-defaults', 'tensorflow==2.0.0', 'opencv-python==4.1.1.26'])

with open('myenv.yml','w') as f:
    f.write(myenv.serialize_to_string())

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory='src',
                                   runtime= 'python', 
                                   entry_script='score.py',
                                   conda_file='myenv.yml')

In [ ]:
from azureml.core.model import Model
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(cpu_cores=2, 
                                                memory_gb=4, 
                                                tags={'model': 'RESNET 50', 'method': 'tensorflow'}, 
                                                description='Predicts NBA player based on image')

aci_service_name = 'nba-player-aciservice'

try:
    aci_service = Webservice(workspace=aci_service_name)
    if aci_service:
        aci_service.delete()
except WebserviceException as e:
    print()

aci_service = Model.deploy(workspace, aci_service_name, [model], inference_config, aci_config)

aci_service.wait_for_deployment(True)
print(aci_service.state)